In [2]:
from google.colab import drive
drive.mount('/content/drive')
print("Loading packages")
!pip install SimpleITK
!pip install sklearn-genetic
import tensorflow as tf
import pywt
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
#import cv2
from tensorflow.keras.layers import Input,Dense,Flatten,Dropout,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization, concatenate, Activation, Add, ZeroPadding2D,LSTM
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Lambda
from tensorflow.keras.optimizers import SGD,Adam
#from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
import SimpleITK as sitk
import numpy as np
import scipy.misc
import numpy.random as rng
#from PIL import Image, ImageDraw, ImageFont
#from sklearn.utils import shuffle
#import nibabel as nib #reading MR images
#from sklearn.cross_validation import train_test_split
#import math
import glob
from skimage.filters.rank import entropy
from genetic_selection import GeneticSelectionCV
from skimage.morphology import disk,rectangle
from sklearn.preprocessing import MinMaxScaler
from skimage.measure import perimeter
from matplotlib import pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from skimage.feature import canny
from skimage.feature import greycomatrix, greycoprops
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,recall_score,f1_score,make_scorer
print("Done")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading packages
     |████████████████████████████████| 48.4 MB 32 kB/s 
  Using cached sklearn_genetic-0.5.1-py3-none-any.whl (11 kB)
     |████████████████████████████████| 160 kB 4.6 MB/s 
Done


In [13]:
def compute_category(df):
  x=[]
  for i in df['Survival']:
    if i < 300:
      x.append(0)
    elif i>450:
      x.append(2)
    else:
      x.append(1)
  return x
df_necrotic = pd.read_csv('drive/My Drive/Necrotic_t1ce_stats_withsurvival.csv')
df_necrotic = df_necrotic.drop('Unnamed: 0',axis=1)
df_enhancing = pd.read_csv('drive/My Drive/Enhancing_t1ce_stats_withsurvival.csv')
df_enhancing = df_enhancing.drop('Unnamed: 0',axis=1)
df_complete = pd.read_csv('drive/My Drive/Complete_t2_stats_withsurvival.csv')
df_complete = df_complete.drop('Unnamed: 0',axis=1)
p=compute_category(df_enhancing)
df_enhancing['Survival_Category']=p
df_img= pd.read_csv('drive/My Drive/img_features.csv')
df_img = df_img.drop('Unnamed: 0',axis=1)
df_wt1=pd.read_csv('drive/My Drive/wt1_features.csv')
df_wt1 = df_wt1.drop('Unnamed: 0',axis=1)
df_wt2=pd.read_csv('drive/My Drive/wt2_features.csv')
df_wt2 = df_wt2.drop('Unnamed: 0',axis=1)
df_img_lbp=pd.read_csv('drive/My Drive/img_lbp_256bin.csv')
df_img_lbp = df_img_lbp.drop('Unnamed: 0',axis=1)

In [16]:
split=130
enhancing_features=df_enhancing.drop(['Brats17ID','Age','Survival','Actual_complete_t2_tumor_volume','Survival_Category'],axis=1).values
necrotic_features=df_necrotic.drop(['Brats17ID','Age','Survival','Actual_complete_t2_tumor_volume'],axis=1).values
complete_features=df_complete.drop(['Brats17ID','Age','Survival','Actual_complete_t2_tumor_volume'],axis=1).values
enhancing_train_features=enhancing_features[:split]
necrotic_train_features=necrotic_features[:split]
complete_train_features=complete_features[:split]
enhancing_test_features=enhancing_features[split:]
necrotic_test_features=necrotic_features[split:]
complete_test_features=complete_features[split:]
img_train_features=df_img.values[:split]
wt1_train_features=df_wt1.values[:split]
wt2_train_features=df_wt2.values[:split]
img_test_features=df_img.values[split:]
wt1_test_features=df_wt1.values[split:]
wt2_test_features=df_wt2.values[split:]
lbp_train_features=df_img_lbp.values[:split]
lbp_test_features=df_img_lbp.values[split:]
age_train=df_enhancing['Age'].values.reshape(-1,1)[:split]
age_test=df_enhancing['Age'].values.reshape(-1,1)[split:]
age=df_enhancing['Age'].values.reshape(-1,1)
y_train = df_enhancing['Survival_Category'].values[:split]
y_test = df_enhancing['Survival_Category'].values[split:]
y=df_enhancing['Survival_Category'].values
print(""+pd.DataFrame(y).value_counts())
print("3 class y test "+pd.DataFrame(y_test).value_counts())
print("Enhancing Train Features: "+str(enhancing_train_features.shape))
print("Enhancing Test Features: "+str(enhancing_test_features.shape))
print("Necrotic Train Features: "+str(necrotic_train_features.shape))
print("Necrotic Test Features: "+str(necrotic_test_features.shape))
print("Complete Train Features: "+str(complete_train_features.shape))
print("Complete Test Features: "+str(complete_test_features.shape))
print("Image Train Features: "+str(img_train_features.shape))
print("Image Test Features: "+str(img_test_features.shape))
print("WT1 Train Features: "+str(wt1_train_features.shape))
print("WT1 Test Features: "+str(wt1_test_features.shape))
print("WT2 Train Features: "+str(wt2_train_features.shape))
print("WT2 Test Features: "+str(wt2_test_features.shape))
print("LBP Train Features "+str(lbp_train_features.shape))
print("LBP Test Features "+str(lbp_test_features.shape))

UFuncTypeError: ignored

In [ ]:
def normalize(x):
    for i in range(x.shape[1]):
        if np.max(x[:,i])!=0:
            x[:,i]=x[:,i]/np.max(x[:,i])
    return x
def appender(necrotic_features,enhancing_features,complete_features,img_features,wt1_features,wt2_features,age,lbp_features):
    X=np.append(necrotic_features,enhancing_features,axis=1)
    X=np.append(X,complete_features,axis=1)
    #X=np.append(X,age,axis=1)
    #X=np.append(X,img_features,axis=1)
    #X=np.append(X,wt1_features,axis=1)
    #X=np.append(X,wt2_features,axis=1)
    #X=np.append(X,lbp_features,axis=1)
    #y=df_enhancing['Survival_Category'].values
    return X

X_train=appender(necrotic_train_features,enhancing_train_features,complete_train_features,img_train_features,wt1_train_features,wt2_train_features,age_train,lbp_train_features)
X_test=appender(necrotic_test_features,enhancing_test_features,complete_test_features,img_test_features,wt1_test_features,wt2_test_features,age_test,lbp_test_features)
X=appender(necrotic_features,enhancing_features,complete_features,df_img.values,df_wt1.values,df_wt2.values,df_enhancing['Age'].values.reshape(-1,1),df_img_lbp.values)
X_test = normalize(X_test.copy())
X_train = normalize(X_train.copy())
X=normalize(X.copy())
print(X.shape)
print(X_test.shape)
print(X_train.shape)

(163, 36)
(33, 36)
(130, 36)


In [ ]:
pca=PCA()
X_pca=pca.fit_transform(X)
X_pca_train=pca.fit_transform(X_train)
X_pca_test=pca.transform(X_test)
X_pca.shape

(163, 36)

In [ ]:
#________________________2PCA__________________________

#__________________Volumetric Feature PCA________________
pca=PCA(n_components=10)
features=np.append(necrotic_features,enhancing_features,axis=1)
features=np.append(features,complete_features,axis=1)
#features=np.append(features,age,axis=1)
features=np.append(features,df_img_lbp.values,axis=1)
features=normalize(features.copy())
pca_features1=pca.fit_transform(features)
#X_pca=np.append(pca_features1,normalize(age),axis=1)
#___________________Deep Feature PCA_____________________

features=np.append(df_img.values,df_wt1.values,axis=1)
features=np.append(features,df_wt2.values,axis=1)
features=normalize(features.copy())
pca_features2=pca.fit_transform(features)
X_pca = np.append(pca_features1,pca_features2,axis=1)
X_pca=np.append(X_pca,normalize(age),axis=1)
print(X_pca.shape)

(163, 21)


In [ ]:
np.append(pca_features1,pca_features2,axis=1).shape

(163, 20)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda=LinearDiscriminantAnalysis(n_components=65)
X_pca_train=lda.fit_transform(X_pca_train,y_train)
X_pca_test=lda.transform(X_pca_test)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 3 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [ ]:
#model =LGBMClassifier(boosting_type='dart',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.15)
model = RandomForestClassifier(n_estimators=500,min_samples_leaf=3,min_samples_split=3,random_state=42)
model.fit(X_pca, y)
y_pred = model.predict(X_pca)
#y_pred_train = model.predict(X_pca_train)
print("LGBM:\n "+str(classification_report(y_pred,y)))
#print(classification_report(y_pred_train,y_train))
f1_score(y_pred,y,average='macro')

LGBM:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99        66
           1       0.98      1.00      0.99        41
           2       1.00      1.00      1.00        56

    accuracy                           0.99       163
   macro avg       0.99      0.99      0.99       163
weighted avg       0.99      0.99      0.99       163



0.9934394064808854

In [ ]:
#_________________________________GENETIC ALGORITHM_______________


estimator=LGBMClassifier(boosting_type='dart',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.15)
#estimator=RandomForestClassifier(n_estimators=100,min_samples_leaf=3,min_samples_split=3,random_state=42)
#scorer=f1_score(average='macro')
selector = GeneticSelectionCV(estimator,
                                  cv=5,
                                  verbose=1,
                                  scoring=make_scorer(f1_score,average='macro'),
                              crossover_proba=0.6,
                              n_population=600,
                                  caching=True,
                              n_generations=10,
                                  n_jobs=1)
selector.fit(X_pca,y)
y_pred=selector.predict(X_pca)
#y_pred_train=selector.predict(X_pca_train)
print("LGBM:\n "+str(classification_report(y_pred,y)))
#print(classification_report(y_pred_train,y_train))

Selecting features with genetic algorithm.
gen	nevals	avg                      	std                    	min                    	max                      
0  	600   	[ 0.38705459 10.57833333]	[0.03686187 2.38967164]	[0.27193919 4.        ]	[ 0.48748611 17.        ]
1  	404   	[ 0.41289398 10.67      ]	[0.03005178 2.42097088]	[0.32348166 3.        ]	[ 0.49349867 19.        ]
2  	395   	[ 0.43147554 10.88      ]	[0.0276704 2.3048355]  	[0.33371144 4.        ]	[ 0.51314893 17.        ]
3  	399   	[ 0.44475076 11.16833333]	[0.02588853 2.14631402]	[0.34951504 4.        ]	[ 0.50098818 16.        ]
4  	405   	[ 0.45448121 11.18      ]	[0.02742042 1.99272008]	[0.34264098 4.        ]	[ 0.51010281 17.        ]
5  	426   	[ 0.46550049 11.19833333]	[0.02580003 1.77923126]	[0.3543186 4.       ]  	[ 0.5174851 16.       ]  
6  	419   	[ 0.47421021 11.08666667]	[0.02391225 1.70269068]	[0.34670172 3.        ]	[ 0.52850521 17.        ]
7  	410   	[ 0.48109331 10.97333333]	[0.0225274  1.66311622]	[0.35377

In [ ]:
selector.scores_cache

{(0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0): 0.3475300486450302,
 (0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0): 0.38375248876796864,
 (0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1): 0.4018633187862397,
 (0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1): 0.3413177809844477,
 (0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1): 0.3283500943500943,
 (0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1): 0.37256852648156996,
 (0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0): 0.30098975438478287,
 (0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1): 0.3336223

In [ ]:
 from genetic_selection import GeneticSelectionCV
estimator=LGBMClassifier(boosting_type='gbdt',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.01)
#scorer=f1_score(average='macro')
 selector=GeneticSelectionCV(estimator,
                                  cv=2,
                                  verbose=1,
                                  scoring=make_scorer(f1_score,average='macro'),
                                  max_features=5,
                                  n_population=50,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=100,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)
 selector.fit(X_pca,y)

In [ ]:
X_pca.shape

(163, 163)

In [ ]:
import numpy as np
from sklearn import datasets, linear_model

from genetic_selection import GeneticSelectionCV


def main():
    iris = datasets.load_iris()

    # Some noisy data not correlated
    E = np.random.uniform(0, 0.1, size=(len(iris.data), 20))

    X = np.hstack((iris.data, E))
    y = iris.target

    estimator = linear_model.LogisticRegression(solver="liblinear", multi_class="ovr")

    selector = GeneticSelectionCV(estimator,
                                  cv=5,
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=5,
                                  n_population=50,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=40,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)
    selector = selector.fit(X, y)

    print(selector.support_)
main()

In [ ]:
from genetic_selection import GeneticSelectionCV
#estimator=LGBMClassifier(boosting_type='dart',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.15)
estimator=RandomForestClassifier(n_estimators=100,min_samples_leaf=3,min_samples_split=3,random_state=42)
#scorer=f1_score(average='macro')
value=120
X_srijib_train=X_pca[:value]
X_srijib_test=X_pca[value:]
y_srijib_train=y[:value]
y_srijib_test=y[value:]
selector = GeneticSelectionCV(estimator,
                                  cv=5,
                                  verbose=1,
                                  scoring=make_scorer(f1_score,average='macro'),
                                  caching=True,
                              n_generations=100,
                                  n_jobs=1)
selector.fit(X_srijib_train,y_srijib_train)
y_pred=selector.predict(X_srijib_test)
#y_pred_train=selector.predict(X_pca_train)
print("LGBM:\n "+str(classification_report(y_pred,y_srijib_test)))
#print(classification_report(y_pred_train,y_train))

Selecting features with genetic algorithm.
gen	nevals	avg                      	std                    	min                    	max                      
0  	300   	[ 0.42060249 10.67333333]	[0.03786519 2.32521445]	[0.31597134 4.        ]	[ 0.51838719 17.        ]
1  	169   	[ 0.45080104 10.79      ]	[0.03125558 2.2329129 ]	[0.3267282 4.       ]  	[ 0.57349521 16.        ]
2  	174   	[ 0.46806246 10.28666667]	[0.02956245 2.25635892]	[0.35999506 4.        ]	[ 0.57349521 16.        ]
3  	172   	[ 0.4836169  10.49333333]	[0.02813417 2.30866965]	[0.3915921 5.       ]  	[ 0.57349521 16.        ]
4  	175   	[ 0.49712081 10.49      ]	[0.0283872  2.36711498]	[0.41023095 5.        ]	[ 0.57349521 17.        ]
5  	169   	[ 0.50679945 10.48666667]	[0.0352423  2.17021248]	[0.38911069 5.        ]	[ 0.57349521 17.        ]
6  	163   	[ 0.52130016 10.37      ]	[0.03700861 1.96462889]	[0.38015505 5.        ]	[ 0.57349521 16.        ]
7  	178   	[0.5282662  9.93333333]  	[0.04092543 1.7230464 ]	[0.39299

In [ ]:
y_srijib_test.shape

(43,)

In [ ]:
X=X_pca.copy()
kf = StratifiedKFold(n_splits=4,shuffle=True,random_state=42)
pred_test_full =0
cv_score =[]
i=1
for train_index,test_index in kf.split(X,y):
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X[train_index],X[test_index]
    ytr,yvl = y[train_index],y[test_index]
    
    #model
    lr = LGBMClassifier(boosting_type='dart',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.15)
    #lr =RandomForestClassifier(n_estimators=100,min_samples_leaf=3,min_samples_split=3,random_state=42)
    lr.fit(xtr,ytr)
    score = f1_score(yvl,lr.predict(xvl),average='macro')
    print('ROC AUC score:',score)
    cv_score.append(score)    
    #pred_test = lr.predict_proba(x_test)[:,1]
    #pred_test_full +=pred_test
    i+=1
print("Avg: "+str(np.average(np.array(cv_score))))

1 of KFold 4
ROC AUC score: 0.4067370537958774
2 of KFold 4
ROC AUC score: 0.28939871045134197
3 of KFold 4
ROC AUC score: 0.5306372549019608
4 of KFold 4
ROC AUC score: 0.3970238095238095
Avg: 0.4059492071682474


In [ ]:
np.average(np.array(cv_score))

0.4030162520707571

In [ ]:
X=X_pca.copy()
kf = StratifiedKFold(n_splits=4,shuffle=True)
pred_test_full =0
cv_score =[]
i=1
for train_index,test_index in kf.split(X,y):
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X[train_index],X[test_index]
    ytr,yvl = y[train_index],y[test_index]
    
    #model
    #lr = LGBMClassifier(boosting_type='dart',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.15)
    #lr =RandomForestClassifier(n_estimators=100,min_samples_leaf=3,min_samples_split=3,random_state=42)
    estimator=LGBMClassifier(boosting_type='gbdt',num_leaves=200,n_estimators=100,colsample_bytree=0.47,max_bin=5000,n_iterations=500000,learning_rate=0.15)
    #estimator=RandomForestClassifier(n_estimators=100,min_samples_leaf=3,min_samples_split=3,random_state=42)
    #scorer=f1_score(average='macro')
    lr = GeneticSelectionCV(estimator,
                                  cv=5,
                                  verbose=1,
                                  scoring=make_scorer(f1_score,average='macro'),
                                  caching=True,
                              n_generations=5,
                                  n_jobs=1)
    lr.fit(xtr,ytr)
    score = f1_score(yvl,lr.predict(xvl),average='macro')
    print('ROC AUC score:',score)
    cv_score.append(score)    
    #pred_test = lr.predict_proba(x_test)[:,1]
    #pred_test_full +=pred_test
    i+=1

1 of KFold 4
Selecting features with genetic algorithm.
gen	nevals	avg                      	std                  	min                    	max                      
0  	300   	[ 0.38994449 10.70333333]	[0.0520526 2.2170526]	[0.22919706 6.        ]	[ 0.52194878 16.        ]
1  	189   	[ 0.4292626  10.95666667]	[0.03772269 2.2718837 ]	[0.31872485 4.        ]	[ 0.52194878 16.        ]
2  	185   	[ 0.45313948 10.51666667]	[0.03280933 2.32301289]	[0.354757 6.      ]    	[ 0.52642226 16.        ]
3  	181   	[0.46965627 9.91666667]  	[0.0286333  2.10310617]	[0.37910596 5.        ]	[ 0.53846288 15.        ]
4  	187   	[0.4834217  9.51666667]  	[0.03041909 1.93641995]	[0.3387478 5.       ]  	[ 0.54996564 16.        ]
5  	191   	[0.49325201 9.53666667]  	[0.0291861  1.83538976]	[0.3697936 5.       ]  	[ 0.54996564 14.        ]
ROC AUC score: 0.3825396825396825
2 of KFold 4
Selecting features with genetic algorithm.
gen	nevals	avg                      	std                    	min                	

In [ ]:
np.average(np.array(cv_score))

0.38530959804617093